In [1]:
# import all the required libraries
from PIL import Image
import numpy as np

def SeparateRGB(image_path):
  image = Image.open(image_path)
  image_array = np.array(image)

  rgb_array = np.transpose(image_array, (2, 0, 1))
  zero_array = np.zeros_like(rgb_array[0])

  r_array = np.stack((rgb_array[0], zero_array, zero_array), axis=0)
  g_array = np.stack((zero_array, rgb_array[1], zero_array), axis=0)
  b_array = np.stack((zero_array, zero_array, rgb_array[2]), axis=0)

  r_array = np.transpose(r_array, (1, 2, 0))
  g_array = np.transpose(g_array, (1, 2, 0))
  b_array = np.transpose(b_array, (1, 2, 0))

  r_image = Image.fromarray(r_array)
  g_image = Image.fromarray(g_array)
  b_image = Image.fromarray(b_array)

  file_name, file_extension = os.path.splitext(image_path)

  r_image_path = file_name + "_r" + file_extension
  g_image_path = file_name + "_g" + file_extension
  b_image_path = file_name + "_b" + file_extension

  r_image.save(r_image_path)
  g_image.save(g_image_path)
  b_image.save(b_image_path)


In [ ]:
# import all the required libraries
import ipywidgets as widgets
from IPython import display
import os

# instantiate the widgets
file_upload = widgets.FileUpload(accept='.jpg', description='Upload Image')
upload_output = widgets.Output()
r_output = widgets.Output()
g_output = widgets.Output()
b_output = widgets.Output()
RGB_box = widgets.HBox([r_output, g_output, b_output])

# save the image in local computer
def on_file_upload(activate):
  uploaded_filename = list(file_upload.value.keys())[0]
  file_content = file_upload.value[uploaded_filename]['content']
  filename = 'image.jpg'
  r_filename = "image_r.jpg"
  g_filename = "image_g.jpg"
  b_filename = "image_b.jpg"

  with open(filename, 'wb') as f:
    f.write(file_content)

  SeparateRGB(filename)

  with upload_output:
    display.clear_output()
    print(f'{uploaded_filename} is uploaded!')
    display.display(display.Image(filename=filename, width=300))

  with r_output:
    display.clear_output()
    display.display(display.Image(filename=r_filename, width=200))

  with g_output:
    display.clear_output()
    display.display(display.Image(filename=g_filename, width=200))

  with b_output:
    display.clear_output()
    display.display(display.Image(filename=b_filename, width=200))

  file_upload.value.clear()
  file_upload._counter = 0

# link the action to the event
file_upload.observe(on_file_upload, names='value')

# display the widgets
display.display(file_upload, upload_output, RGB_box)

In [3]:
# import all the required libraries
import numpy as np

# apply histogram equalization to a 2-d metrix
# input: image metrix (W*H ndarray)
# output: image metrix after histogram equalization (W*H ndarray)
def HisEqual(image_array):
  BIT_DEPTH = 256
  PIXEL_NUM = image_array.size

  new_image = np.zeros_like(image_array)
  intensity_proportion = np.zeros(BIT_DEPTH)

  for pixel in image_array.flat:
    intensity_proportion[pixel] += 1

  intensity_proportion = intensity_proportion.cumsum()

  for i in range(BIT_DEPTH):
    intensity_proportion[i] = round(intensity_proportion[i]/PIXEL_NUM * (BIT_DEPTH-1))

  for i in range(image_array.shape[0]):
    for j in range(image_array.shape[1]):
      pixel = image_array[i][j]
      new_image[i][j] = intensity_proportion[pixel]

  return new_image


In [4]:
# Translate an RGB image metrix to an HSL image metrix
# input: RGB image metrix (W*H*3 ndarray)
# output: HSL image metrix (W*H*3 ndarray)
def RGB2HSL(rgb_image_array):
  BIT_DEPTH = 256

  '''
  NOTE: If the type of the ndarray is not specified,
    the type of the generated ndarray will be the same as rgb_image_array,
    i.e., int. This can lead to rounding errors in the calculation.
  '''
  hsl_image_array = np.zeros_like(rgb_image_array, dtype=float)

  for i in range(rgb_image_array.shape[0]):
    for j in range(rgb_image_array.shape[1]):
      r = rgb_image_array[i][j][0]/(BIT_DEPTH-1)
      g = rgb_image_array[i][j][1]/(BIT_DEPTH-1)
      b = rgb_image_array[i][j][2]/(BIT_DEPTH-1)
      max = np.max(rgb_image_array[i][j]) / (BIT_DEPTH-1) # Note: Don't forget to divide by (BIT_DEPTH-1)
      min = np.min(rgb_image_array[i][j]) / (BIT_DEPTH-1)

      delta = max-min

      hsl_image_array[i][j][2] = (max+min)/2

      if abs(delta)<1e-7:  # Note: Use abs(float)<1e-7 rather than float==0
        continue

      '''
      Note: The way to calculate H is slightly different
        from the one on the website (https://www.rapidtables.com/convert/color/rgb-to-hsl.html),
        but I think the formula given in the code is more reasonable.
      '''
      if abs(max-r)<1e-7:
          hsl_image_array[i][j][0] = (60 * ((g - b) / delta)) % 360
      elif abs(max-g)<1e-7:
          hsl_image_array[i][j][0] = (60 * ((b - r) / delta) + 120) % 360
      elif abs(max-b)<1e-7:
          hsl_image_array[i][j][0] = (60 * ((r - g) / delta) + 240) % 360

      hsl_image_array[i][j][1] = delta / (1 - abs(max+min-1))

  return hsl_image_array

# Translate an HSL image metrix to an RGB image metrix
# input: HSL image metrix (W*H*3 ndarray)
# output: RGB image metrix (W*H*3 ndarray)
def HSL2RGB(hsl_image_array):
  BIT_DEPTH = 256

  rgb_image_array = np.zeros_like(hsl_image_array)

  for i in range(hsl_image_array.shape[0]):
    for j in range(hsl_image_array.shape[1]):
      h = hsl_image_array[i][j][0]
      s = hsl_image_array[i][j][1]
      l = hsl_image_array[i][j][2]
      c = (1 - abs(2*l-1)) * s
      x = c * (1 - abs((h/60) % 2 - 1))
      m = l - c/2

      r = 0
      g = 0
      b = 0

      if 0 <= h < 60:
        r = c
        g = x
      elif 60 <= h < 120:
        r = x
        g = c
      elif 120 <= h < 180:
        g = c
        b = x
      elif 180 <= h < 240:
        g = x
        b = c
      elif 240 <= h < 300:
        r = x
        b = c
      elif 300 <= h < 360:
        r = c
        b = x

      '''
      Note: HSL is float matrix, RGB is unsigned int matrix.
        So, don't forget to use round() and astype(np.uint8).
      '''
      rgb_image_array[i][j][0] = round((r+m) * (BIT_DEPTH-1))
      rgb_image_array[i][j][1] = round((g+m) * (BIT_DEPTH-1))
      rgb_image_array[i][j][2] = round((b+m) * (BIT_DEPTH-1))

  return rgb_image_array.astype(np.uint8)


In [5]:
# import all the required libraries
from PIL import Image

# Apply histogram equalization to the image at image_path, then save
# the equalized gray image to gray_image_path.
def GrayImageEqual(image_path, gray_image_path):
  image = Image.open(image_path)
  gray_image = image.convert("L")
  gray_array = np.array(gray_image)
  image.close()

  he_gray_array = HisEqual(gray_array)
  he_gray_image = Image.fromarray(he_gray_array)
  he_gray_image.save(gray_image_path)

# Apply histogram equalization channel by channel to the image at image_path,
# then save the equalized RGB image to rgb_image_path.
def RGBImageEqual(image_path, rgb_image_path):
  image = Image.open(image_path)
  rgb_image = image.convert("RGB")
  rgb_array = np.array(rgb_image)
  image.close()

  he_rgb_array = []
  rgb_array = np.transpose(rgb_array, (2, 0, 1))  # Note: Permute the axes, so the computation would become easier.
  for channel in rgb_array:
    he_rgb_array.append(HisEqual(channel))
  he_rgb_array = np.transpose(he_rgb_array, (1, 2, 0))

  he_rgb_image = Image.fromarray(he_rgb_array)
  he_rgb_image.save(rgb_image_path)

# Convert the RGB image at image_path to HSL image, apply histogram
# equalization to the L channel, then convert it to RGB image again.
# Save the equalized image to hsl_image_path.
def HSLImageEqual(image_path, hsl_image_path):
  image = Image.open(image_path)
  rgb_image = image.convert("RGB")
  rgb_array = np.array(rgb_image)
  image.close()

  hsl_array = RGB2HSL(rgb_array)

  he_hsl_array = []

  hsl_array = np.transpose(hsl_array, (2, 0, 1))
  he_hsl_array.append(hsl_array[0])
  he_hsl_array.append(hsl_array[1])
  he_hsl_array.append(HisEqual((hsl_array[2]*255).astype(np.uint8)) / 255)
  he_hsl_array = np.transpose(he_hsl_array, (1, 2, 0))

  he_hsl_image = Image.fromarray(HSL2RGB(he_hsl_array))
  he_hsl_image.save(hsl_image_path)

In [ ]:
# import all the required libraries
import ipywidgets as widgets
from IPython import display

# instantiate the widgets
button_1 = widgets.Button(description='Gray Image Histogram Equalization', layout=widgets.Layout(width='300px'))
button_2 = widgets.Button(description='RGB Image Histogram Equalization', layout=widgets.Layout(width='300px'))
button_3 = widgets.Button(description='HSL Image Histogram Equalization', layout=widgets.Layout(width='300px'))
equalized_output = widgets.Output()

# show the equalized gray image
def on_button_1_click(activate):
  image_path = 'image.jpg'
  gray_image_path = 'gray_image.jpg'
  GrayImageEqual(image_path, gray_image_path)

  with equalized_output:
    display.clear_output()
    print('The image is equalized in a gray image way!')
    display.display(display.Image(filename=gray_image_path, width=300))

# show the equalized RGB image
def on_button_2_click(activate):
  image_path = 'image.jpg'
  rgb_image_path = 'rgb_image.jpg'
  RGBImageEqual(image_path, rgb_image_path)

  with equalized_output:
    display.clear_output()
    print('The image is equalized in an RGB image way!')
    display.display(display.Image(filename=rgb_image_path, width=300))

# show the equalized HSL image
def on_button_3_click(activate):
  image_path = 'image.jpg'
  hsl_image_path = 'hsl_image.jpg'
  HSLImageEqual(image_path, hsl_image_path)

  with equalized_output:
    display.clear_output()
    print('The image is equalized in an HSL image way!')
    display.display(display.Image(filename=hsl_image_path, width=300))

# link the action to the event
button_1.on_click(on_button_1_click)
button_2.on_click(on_button_2_click)
button_3.on_click(on_button_3_click)

# display the widgets
display.display(button_1, button_2, button_3, equalized_output)